# Primer modelo: red neuronal sin preprocesamiento de datos

## Tabla de contenidos 

1. [Contexto del modelo](#id1)

    1.1. [Librerías utilizadas](#id2)

2. [Construcción y preprocesado del dataset](#id3)

    2.1. [Text Vectorization](#id4)
    
3. [Construcción del modelo](#id5)

    3.1. [Explicación de las distintas capas del modelo](#id6) 

    3.2. [Explicación de la compilación del modelo](#id7)
    
    3.3. [Explicación del entrenamiento](#id8)

4. [Predicciones del modelo](#id9)

    4.1. [Predicciones de prueba](#id10)

    4.2. [Predicciones en bloque](#id11)

5. [Evaluación del modelo](#id12)

6. [Exportación del modelo](#id13)

<div id='id1' />


## 1. Contexto del modelo

Este primer modelo se presenta como punto de referencia para futuras comparaciones. Se ha entrenado utilizando el conjunto de datos sin ningún tipo de preprocesamiento previo. por lo que se espera que al comparar los resultados de este modelo con los de otros que sí implementan técnicas de limpieza de datos, pueda observarse el impacto de dichas técnicas en la precisión de la clasificación.

El modelo en cuestión es una red neuronal recurrente (RNN) con una estructura bidireccional LSTM (Long Short-Term Memory), diseñada para la tarea de clasificación multietiqueta.

<div id='id2' />


## 1.1. Librerías utilizadas

In [39]:
# Tratamiento de datos

import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

# Gráficas

import seaborn as sns
import matplotlib as mpl 
import matplotlib.pyplot as plt

# Preprocesamiento y modelado 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import TextVectorization, LSTM, Dropout, Bidirectional, Dense, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

<div id='id3' />


## 2. Construcción y preprocesado del dataset

In [2]:
comments_raw_df = pd.read_csv('../01_data/01_raw/comments_raw.csv', delimiter=',')

In [3]:
comments_raw_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
comments_raw_df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [5]:
comments_raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [6]:
# Ejemplo de un comentario

comments_raw_df.iloc[2]['comment_text']

"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."

In [7]:
comments_raw_df.iloc[6]

id                                           0002bcb3da6cb337
comment_text     COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
toxic                                                       1
severe_toxic                                                1
obscene                                                     1
threat                                                      0
insult                                                      1
identity_hate                                               0
Name: 6, dtype: object

In [8]:
comments_raw_df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [9]:
total_sum_short = comments_raw_df[['toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate']].sum().sum().sum().sum().sum().sum()
print(total_sum_short)

35098


In [10]:
toxic_sum= comments_raw_df['toxic'].sum()
toxic_sum

15294

In [11]:
severe_toxic_sum= comments_raw_df['severe_toxic'].sum()
severe_toxic_sum

1595

In [12]:
obscene_sum= comments_raw_df['obscene'].sum()
obscene_sum

8449

In [13]:
threat_sum= comments_raw_df['threat'].sum()
threat_sum

478

In [14]:
insult_sum= comments_raw_df['insult'].sum()
insult_sum

7877

In [15]:
identity_hate_sum = comments_raw_df['identity_hate'].sum()
identity_hate_sum

1405

In [40]:
total_sum = identity_hate_sum + insult_sum + threat_sum + obscene_sum + severe_toxic_sum + toxic_sum

total_sum

35098

<div id='id4' />


### 2.1. Text Vectorization

Text vectorization: capa de preprocesamiento que convierte las características del texto en secuencias de números enteros.


In [18]:
X = comments_raw_df['comment_text'] # guarda los comentarios como serie de pandas

y = comments_raw_df[comments_raw_df.columns[2:]].values # transformamos las etiquetas en un array de 2 dimensiones

In [19]:
X.shape

(159571,)

In [20]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [21]:
# Creamos la text vectorization layer

max_features = 200000 # número de palabras en el diccionario de consulta. Cuanto más alto sea este número más preciso será el modelo (aunque más recursos necesitará para funcionar)

vectorizer = TextVectorization(max_tokens= max_features,
                               output_sequence_length= 1800, # longitud máxima en palabras (tokens) de cada frase del dataframe que puede procesar 
                               output_mode= 'int')

In [22]:
vectorizer.adapt(X.values) # esta función le enseña al vectorizer todas las palabras que hay en los comentarios para que se las aprenda

In [23]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [24]:
vectorizer.vocabulary_size()

200000

In [25]:
vectorized_text = vectorizer(X.values)

In [26]:
vectorized_text 

# vemos que cada palabra de cada comentario ha sido asociada a un número entero y ahora tenemos una representación numérica de cada comentario
# cuando los comentarios no llegan al máximo de 1800 palabras que hemos colocado en los parámetros completa el resto con ceros

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [27]:
# generamos el pipeline con el que trabaja tensor flow

dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y)) # Se crea un "tf.data.Dataset" a partir de los tensores "vectorized_text" (features) y "y" (targets)
dataset = dataset.cache() # Los datos se almacenan en memoria después de la primera pasada, lo que acelera el proceso en las sucesivas iteraciones 
dataset = dataset.shuffle(160000) # Los datos se mezclan aleatoriamente.
dataset = dataset.batch(16) # Los datos se agrupan en lotes de tamaño 16
dataset = dataset.prefetch(8) # Los siguientes 8 lotes se preparan mientras el modelo está entrenando con el lote actual para así evitar cuellos de botella

In [28]:
# batch de ejemplo para ver que el código anterior se ha ejecutado correctamente

batch_X, batch_y = dataset.as_numpy_iterator().next()

In [29]:
batch_X # el texto del un batch en su forma vectorial 

array([[   171, 157645,      8, ...,      0,      0,      0],
       [    20,    319,  19139, ...,      0,      0,      0],
       [ 38966,     25,    914, ...,      0,      0,      0],
       ...,
       [  2303,   2048,     28, ...,      0,      0,      0],
       [     1,  17670,    570, ...,      0,      0,      0],
       [    40,   1562,     16, ...,      0,      0,      0]])

In [30]:
batch_y # las etiquetas de un batch

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [31]:
batch_X.shape

(16, 1800)

In [32]:
batch_y.shape

(16, 6)

In [36]:
train = dataset.take(int(len(dataset)*0.7)) # calcula el 70% del tamaño total del dataset y nos devuelve un nuevo dataset (take) que contiene los n primeros elementos del dataset original
val = dataset.skip(int(len(dataset)*0.7)).take(int(len(dataset)*0.2)) # hace lo mismo que el anterior pero con el "skip" que sirve para que se salte los primeros 70 y 90% de los elementos, respectivamente
test = dataset.skip(int(len(dataset)*0.9)).take(int(len(dataset)*0.1))

<div id='id5' />

## 3. Construcción del modelo

In [24]:
max_len = 1800

# instanciamos el modelo

model = Sequential()

# creamos la red neuronal con sus respectivas capa

# embeeding layer (capa de entrada)
model.add(Embedding(input_dim= max_features + 1, output_dim= 32, input_shape = (max_len,)))

#bidirectional LSTM layer (capa intermedia)
model.add(Bidirectional(LSTM(32, activation= 'tanh')))

# fully connected layer (capa intermedia)
model.add(Dense(128, activation= 'relu'))
model.add(Dense(256, activation= 'relu'))
model.add(Dense(128, activation= 'relu'))

# final layer (capa de salida)
model.add(Dense(6, activation= 'sigmoid'))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


<div id='id6' />

### 3.1. Explicación de las distintas capas del modelo

1. Capa de embedding:
    * Embedding es una capa que convierte índices enteros (que representan palabras) en vectores densos de tamaño fijo.
    * max_features + 1 es el tamaño del vocabulario más uno (para incluir un token de padding).
    * 32 es la dimensión del espacio de embedding en el que se mapearán las palabras.

2. Capa LSTM bidireccional:

    * Bidirectional es una envoltura para capas recurrentes (como LSTM) que hace que la red procese la secuencia de entrada en ambas direcciones (hacia adelante y hacia atrás). Con ello conseguimos que se entienda el contexto de la oración.
    * LSTM (Long Short-Term Memory) es un tipo especial de red neuronal recurrente (RNN) diseñado para manejar problemas de aprendizaje a largo plazo y evitar el problema del "desvanecimiento del gradiente" que puede ocurrir en las RNNs tradicionales.
        * LSTM(32, activation='tanh') es una capa LSTM con 32 unidades ocultas y una función de activación tanh.
    * Esta capa permite capturar dependencias tanto hacia adelante como hacia atrás en las secuencias de texto.

3. Capas densas (fully connected): 

    * Dense es una capa completamente conectada (fully connected) que conecta cada unidad de la capa anterior con cada unidad de la capa actual.
    * 128 y 256 son los números de unidades en cada capa densa.
    * activation='relu' es la función de activación ReLU (Rectified Linear Unit) que se utiliza para introducir no linealidades en el modelo. 

4. Capa de salida:

    * Dense(6, activation='sigmoid') es la capa de salida con 6 unidades, cada una correspondiente a una de las etiquetas de toxicidad.
    * activation='sigmoid' se utiliza porque estamos haciendo una clasificación multietiqueta. La función sigmoide devolverá un valor entre 0 y 1 para cada etiqueta, interpretándose como la probabilidad de pertenecer a esa etiqueta.

In [25]:
# compilamos el modelo

model.compile(loss= 'BinaryCrossentropy', optimizer= 'Adam')

<div id='id7' />

### 3.2. Explicación de la compilación del modelo 

- "model.compile" es el método que configura el modelo para el entrenamiento.
- En este método, especificamos dos cosas principales: la función de pérdida ("loss") y el optimizador ("optimizer").

**Función de Pérdida:**

   - La función de pérdida, o "loss", mide la discrepancia entre las predicciones del modelo y los valores reales.
      - BinaryCrossentropy'` se usa cuando estamos tratando con problemas de clasificación binaria o multietiqueta (multi-label classification) donde cada etiqueta se trata como una clasificación binaria.
   
**Optimizador:**

   - El optimizador ajusta los pesos del modelo en función de la función de pérdida calculada.
      - "Adam" es un optimizador popular que combina las ventajas de los algoritmos "AdaGrad" y "RMSProp".
         - Adam (Adaptive Moment Estimation) utiliza dos momentos (la media y la varianza) para adaptar las tasas de aprendizaje de cada parámetro. Esto permite una convergencia más rápida y estable.
         - Los hiperparámetros clave de Adam son:
            - "learning_rate": Tasa de aprendizaje (por defecto es 0.001).
            - "beta_1": El parámetro de decaimiento exponencial para el primer momento (por defecto es 0.9).
            - "beta_2": El parámetro de decaimiento exponencial para el segundo momento (por defecto es 0.999).
            - "epsilon": Un pequeño valor para evitar la división por cero (por defecto es 1e-7).

In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1800, 32)       │     6,400,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64)             │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,491,686 (24.76 MB)

 Trainable params: 6,491,686 (24.76 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
# entrenamos el modelo

history = model.fit(train, epochs=1, validation_data=val)

6981/6981 ━━━━━━━━━━━━━━━━━━━━ 4105s 587ms/step - loss: 0.0837 - val_loss: 0.0490


<div id='id8' />

### 3.3. Explicación del entrenamiento 

**Argumentos**:

- train: Este es el conjunto de datos de entrenamiento que se pasa al método fit(). Debe ser un objeto de tipo tf.data.Dataset que contiene lotes de datos de entrada (X) y sus correspondientes etiquetas (y).

- epochs=1: Especifica el número de veces que el modelo iterará sobre el conjunto de datos de entrenamiento completo durante el entrenamiento. En este caso, el modelo se entrenará durante una sola época (una pasada completa a través de todos los datos de entrenamiento).

- validation_data=val: Este parámetro opcional proporciona datos de validación para monitorear el rendimiento del modelo en un conjunto de datos separado durante el entrenamiento. val debe ser también un objeto de tipo tf.data.Dataset que contiene datos de entrada y sus etiquetas de validación.

<div id='id9' />

# 4. Predicciones

<div id='id10' />

### 4.1. Predicciones de prueba

In [30]:
input_text = vectorizer('You are a piece of shit!')
input_text

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([ 7, 20,  6, ...,  0,  0,  0])>

In [37]:
np.expand_dims(input_text,0)

array([[ 7, 20,  6, ...,  0,  0,  0]])

In [32]:
prediction = model.predict(np.expand_dims(input_text,0))
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


array([[0.9893178 , 0.18039057, 0.8986939 , 0.0452309 , 0.7611507 ,
        0.10763187]], dtype=float32)

In [37]:
comments_raw_df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [43]:
input_text_2 = vectorizer('You freaking suck! I am going to kill you')

In [44]:
input_text_2

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([   7, 7158,  397, ...,    0,    0,    0])>

In [45]:
prediction_2 = model.predict(np.expand_dims(input_text_2,0))
prediction_2

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


array([[0.99515516, 0.29082724, 0.9413193 , 0.05430832, 0.8366643 ,
        0.13193467]], dtype=float32)

<div id='id11' />

### 4.2. Predicciones en bloque

In [47]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [49]:
prediction_3 = (model.predict(batch_X) > 0.5).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step


In [50]:
prediction_3

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [38]:
comments_raw_df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [51]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

<div id='id12' />

# 5. Evaluación del modelo

In [55]:
precision = Precision()
recall = Recall()
categorical_accuracy = CategoricalAccuracy()

In [60]:
# generamos un bucle para que realice una predicción de todos los batches que forman el set de test

for batch in test.as_numpy_iterator():
    X_test, y_test = batch
    y_pred = model.predict(X_test)

    y_test = y_test.flatten()
    y_pred = y_pred.flatten()

    precision.update_state(y_test, y_pred)
    recall.update_state(y_test, y_pred)
    categorical_accuracy.update_state(y_test, y_pred)

print(f'Precision: {precision.result().numpy()} \n')
print(f'Recall: {recall.result().numpy()}\n')
print(f'Categorical accuracy: {categorical_accuracy.result().numpy()}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

2024-07-05 07:41:05.270835: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Precision: 0.7780060172080994 

Recall: 0.7013245820999146

Categorical accuracy: 0.47342026233673096


Los resultados sugieren que la red neuronal tiene un buen rendimiento general en la clasificación de comentarios tóxicos. La precisión y la recuperación son relativamente altas, lo que significa que la red neuronal es capaz de identificar correctamente la mayoría de los comentarios tóxicos y evitar clasificar erróneamente comentarios no tóxicos como tóxicos. Sin embargo, la precisión categórica es más baja, lo que indica que la red neuronal tiene más dificultad para distinguir entre algunas de las categorías más específicas.

* **Precision** (Precisión): Mide la fracción de predicciones positivas correctas entre todas las predicciones positivas realizadas por el modelo.

* **Recall** (Recuperación): Mide la fracción de instancias positivas que fueron correctamente detectadas por el modelo.

* **Categorical Accuracy** (Exactitud Categórica): Mide la exactitud de las predicciones del modelo en términos de la coincidencia exacta entre las etiquetas verdaderas y las predicciones.

<div id='id11' />

# 6. Exportación del modelo

In [64]:
# Guardamos el modelo como archivo .keras

model_path = '../04_models/primer_modelo.keras'
model.save('../04_models/primer_modelo.keras')

# Diccionario con la ruta del modelo, el vectorizador y las métricas

objects_to_save = {
    'model_path': model_path,
    'vectorizer': vectorizer,
    'precision': precision,
    'recall': recall,
    'categorical_accuracy': categorical_accuracy}

# Guardar el diccionario en un archivo .pkl

with open('model_and_objects.pkl', 'wb') as f:
    pickle.dump(objects_to_save, f)
